In [2]:
# conda 25.5.1
# python 3.10.10
# torch 2.2.2
# transformers 4.38.2
# accelerate 0.30.0
# peft 0.10.0 (使用旧版本，与 transformers 4.38.2 兼容)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType

# select a model
BASE_MODEL_NAME = "uer/gpt2-chinese-cluecorpussmall"

# AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

# AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_NAME)

lora_config = LoraConfig(
    r=8,    # lower rank for less memory usage
    lora_alpha=32,
    target_modules=[
        "c_attn", "c_proj",
        "mlp.c_fc", "mlp.c_proj",
    ],  # 只对注意力层和 MLP 层进行 LoRA 微调
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(model, lora_config)

# use peft to configure LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


/Users/jiaronghe/Desktop/projects/light-weight-private-llm/chapter4/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jiaronghe/Desktop/projects/light-weight-private-llm/chapter4/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


trainable params: 1,179,648 || all params: 103,248,384 || trainable%: 1.1425341049405675


/Users/jiaronghe/Desktop/projects/light-weight-private-llm/chapter4/.venv/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1059: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [2]:
# check device
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")


Using device: mps


In [6]:
from datasets import load_dataset

BLOCK_SIZE = 128
TRAIN_FILE = "cleaned_huagaiji.txt"

# load raw dataset
raw_dataset = load_dataset("text", data_files={"train": TRAIN_FILE})

# define function to tokenize the dataset
def tokenize_function(samples):
    return tokenizer(samples["text"])

# tokenize the dataset
# batched=True 以批处理方式进行分词，加快处理速度
# remove_columns=["text"] 移除原始文本列, 只保留 tokenized 后的数据
tokenized_dataset = raw_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],
)

# 定义一个函数将文本分块
# 例如： BLOCK_SIZE = 128, 那么每块将包含128个token
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // BLOCK_SIZE) * BLOCK_SIZE
    # Split by chunks of max_len
    result = {
        k: [t[i : i + BLOCK_SIZE] for i in range(0, total_length, BLOCK_SIZE)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

# 使用 group_texts 函数将文本分块
lm_dataset = tokenized_dataset.map(
    group_texts,
    batched=True,
    batch_size=1000,
)

print(lm_dataset)

Generating train split: 1139 examples [00:00, 231548.67 examples/s]
Map: 100%|██████████| 1139/1139 [00:00<00:00, 3902.13 examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 461
    })
})


In [7]:
from transformers import TrainingArguments

OUTPUT_DIR = "./gpt2-luxun-finetuned-mps"
NUM_TRAIN_EPOCHS = 3
PER_DEVICE_TRAIN_BATCH_SIZE = 4
LEARNING_RATE = 5e-5
SAVE_STEPS = 500
OVERWRITE_OUTPUT_DIR = True

# 设置训练参数
# 例如： num_train_epochs 可以设置为你希望的训练轮数
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=OVERWRITE_OUTPUT_DIR,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    save_steps=SAVE_STEPS,
    learning_rate=LEARNING_RATE,
    logging_steps=50, # 每50步打印一次日志
    evaluation_strategy="steps", # 每500步评估一次
    eval_steps=500,
    save_total_limit=2, # 最多保存2个checkpoint
    warmup_ratio=0.03, # 预热比率
    use_mps_device=True, # 使用MPS设备
)


/Users/jiaronghe/Desktop/projects/light-weight-private-llm/chapter4/.venv/lib/python3.10/site-packages/transformers/training_args.py:1951: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(


In [8]:
from transformers import Trainer, DataCollatorForLanguageModeling

# 使用 DataCollatorForLanguageModeling 作为数据整理器
# 在构建每个batch时，自动进行动态填充 padding, 掩码生成 masking, 以及标签对齐
# 这里的mlm=False, 表示我们不使用掩码语言模型 (masked language modeling) 进行训练
# 因为我们在做自回归语言模型 (causal language modeling) 训练
# 如果你使用的是 BERT 模型, 那么 mlm=True, 表示使用掩码语言模型进行训练

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# 创建 Trainer 对象
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("--- 开始训练 ---")
trainer.train()
print("--- 训练完成 ---")


/Users/jiaronghe/Desktop/projects/light-weight-private-llm/chapter4/.venv/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


--- 开始训练 ---


 15%|█▍        | 51/348 [00:09<00:31,  9.29it/s]

{'loss': 4.1968, 'grad_norm': 1.2414604425430298, 'learning_rate': 4.421364985163205e-05, 'epoch': 0.43}


 29%|██▉       | 101/348 [00:14<00:25,  9.54it/s]

...

{'loss': 3.7739, 'grad_norm': 1.4555388689041138, 'learning_rate': 1.454005934718101e-05, 'epoch': 2.16}


 86%|████████▋ | 301/348 [00:37<00:05,  9.15it/s]

{'loss': 3.7541, 'grad_norm': 1.8545022010803223, 'learning_rate': 7.12166172106825e-06, 'epoch': 2.59}


100%|██████████| 348/348 [00:42<00:00,  8.19it/s]

{'train_runtime': 42.4751, 'train_samples_per_second': 32.56, 'train_steps_per_second': 8.193, 'train_loss': 3.8502285836756913, 'epoch': 3.0}
--- 训练完成 ---


In [9]:
# 保存模型
print(f"--- 保存模型: {OUTPUT_DIR} ---")
model.save_pretrained(OUTPUT_DIR)

# 保存 tokenizer
tokenizer.save_pretrained(OUTPUT_DIR)
print("--- 完成 ---")

--- 保存模型: ./gpt2-luxun-finetuned-mps ---
--- 完成 ---


In [11]:
from peft import PeftModel

MODEL_PATH = "./gpt2-luxun-finetuned-mps"
BASE_MODEL_NAME = "uer/gpt2-chinese-cluecorpussmall"

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

print("--- 加载分词器 ---")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

print("--- 加载基础模型 ---")
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_NAME)

print("--- 加载 LoRA adapter ---")
model = PeftModel.from_pretrained(base_model, MODEL_PATH)
model.to(device)
print("--- 模型加载完成 ---")


Using device: mps
--- 加载分词器 ---
--- 加载基础模型 ---
--- 加载 LoRA adapter ---
--- 模型加载完成 ---


In [14]:
def generate_text(prompt_text, max_length=150):

    print(f"--- 生成文本: {prompt_text} ---")
    # 编码：将起始的句子转换为模型可以理解的id，并移动到设备
    inputs = tokenizer(prompt_text, return_tensors="pt").to(device)

    # 生成文本，调用模型的generate方法
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1, # 生成几个不同的结果
        do_sample=True, # 是否使用采样, 让文本更具有创造性
        top_k=50, # 从概率最高的50个词中选择
        top_p=0.95, # 从概率总和为95%的词汇中选择
        temperature=0.9, # 控制生成随机性，值越大，随机性越高
        repetition_penalty=1.5, # 控制重复词的出现，值越大，重复词出现的概率越低
    )

    # 解码：将模型生成的id转换为文本
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    print("--- 生成结果 ---")
    print(generated_text)

prompt_text = "秋天的后半夜，月亮下去了，太阳还没有出，只剩下一片乌蓝的天；"
generate_text(prompt_text, max_length=300)



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


--- 生成文本: 秋天的后半夜，月亮下去了，太阳还没有出，只剩下一片乌蓝的天； ---
--- 生成结果 ---
秋 天 的 后 半 夜 ， 月 亮 下 去 了 ， 太 阳 还 没 有 出 ， 只 剩 下 一 片 乌 蓝 的 天 ； 秋
